In [1]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=958f00234a2d0c5abb68b959e99b7b2e15090e51dc2db8cdc520a693f85a268a
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji


In [2]:
!pip install --user unicodedata2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 10.4 MB/s eta 0:00:00


In [3]:
!pip install wordsegment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.3 MB/s eta 0:00:00


## Import Statements

In [38]:
import json
import csv
import re
import emoji
from wordsegment import load, segment
import unicodedata
import pandas as pd
import nltk
from nltk.corpus import stopwords
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np
from scipy import stats
from ast import literal_eval
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
from transformers import RobertaModel, RobertaConfig
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:

# Load word segment data
load()
# Function to clean text
def clean_text(text):
    # Remove usernames
    text = re.sub(r'@[\w]+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Replace emojis with their unicode description
    text = emoji.demojize(text, language='alias')
    # Hashtag segmentation
    text = re.sub(r'#(\w+)', lambda m: ' '.join(segment(m.group(1))), text)
    # Remove links
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove unwanted '/'
    text = re.sub(r'/\s', '', text)
    # Remove punctuation except ' - / % & ... ¿ ?
    text = re.sub(r'[^\w\s\'%&¿?/-]', '', text)
    # Remove accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text
# Load JSON data
with open('/content/EXIST2023_training.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
# Extract data and clean tweets
rows = []
for key in data:
    item = data[key]
    row = [
        item['id_EXIST'],
        item['lang'],
        clean_text(item['tweet']),
        item['number_annotators'],
        item['gender_annotators'],
        item['age_annotators'],
        item['labels_task1']
    ]
    rows.append(row)
# Write data to CSV
with open('data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['id_EXIST', 'lang', 'tweet', 'number_annotators', 'gender_annotators', 'age_annotators', 'labels_task1'])
    csvwriter.writerows(rows)

In [5]:
df_data = pd.read_csv("data.csv")

In [6]:
df_data.head()

,id_EXIST,lang,tweet,number_annotators,gender_annotators,age_annotators,labels_task1
0,100001,es,ignora al otro es un capulloel problema con e...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['YES', 'YES', 'NO', 'YES', 'YES', 'YES']"
1,100002,es,si comicsgate se parece en algo a gamergate p...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'YES', 'NO']"
2,100003,es,lee sobre gamergate y como eso ha cambiado la...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'NO', 'NO']"
3,100004,es,un retraso social bastante lamentable gamerga...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'YES', 'NO', 'YES', 'YES']"
4,100005,es,entonces como asi es el mercado lo mejor no...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['YES', 'NO', 'YES', 'NO', 'YES', 'YES']"


### Remove stopwords from spanish tweets

In [7]:
nltk.download('stopwords')

# Remove stopwords from Spanish tweets
spanish_stopwords = stopwords.words('spanish')
df_data.loc[df_data['lang'] == 'es', 'tweet'] = df_data.loc[df_data['lang'] == 'es', 'tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in spanish_stopwords]))

# Remove stopwords from English tweets
english_stopwords = stopwords.words('english')
df_data.loc[df_data['lang'] == 'en', 'tweet'] = df_data.loc[df_data['lang'] == 'en', 'tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in english_stopwords]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
df_data.head()

,id_EXIST,lang,tweet,number_annotators,gender_annotators,age_annotators,labels_task1
0,100001,es,ignora capulloel problema youtuber denuncia ac...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['YES', 'YES', 'NO', 'YES', 'YES', 'YES']"
1,100002,es,si comicsgate parece gamergate pues bien acoso...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'YES', 'NO']"
2,100003,es,lee gamergate cambiado manera comunicamos inte...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'NO', 'NO']"
3,100004,es,retraso social bastante lamentable gamergate t...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'YES', 'NO', 'YES', 'YES']"
4,100005,es,entonces asi mercado mejor hacer cambiarlo seg...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['YES', 'NO', 'YES', 'NO', 'YES', 'YES']"


In [9]:
# function to remove data rows from dataframe if "yes" and "no" labels for each tweet are tied in label_task1 column
def filter_tied_rows(row):
    yes_count = row['labels_task1'].count('YES')
    no_count = row['labels_task1'].count('NO')
    
    if yes_count == no_count:
        return False
    return True

df_cleaned = df_data[df_data.apply(filter_tied_rows, axis=1)]

In [10]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6920 entries, 0 to 6919
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_EXIST           6920 non-null   int64 
 1   lang               6920 non-null   object
 2   tweet              6920 non-null   object
 3   number_annotators  6920 non-null   int64 
 4   gender_annotators  6920 non-null   object
 5   age_annotators     6920 non-null   object
 6   labels_task1       6920 non-null   object
dtypes: int64(2), object(5)
memory usage: 378.6+ KB


In [11]:
df_cleaned.head()
#df_cleaned.to_csv('df_cleaned.csv', index=False)

,id_EXIST,lang,tweet,number_annotators,gender_annotators,age_annotators,labels_task1
0,100001,es,ignora capulloel problema youtuber denuncia ac...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['YES', 'YES', 'NO', 'YES', 'YES', 'YES']"
1,100002,es,si comicsgate parece gamergate pues bien acoso...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'YES', 'NO']"
2,100003,es,lee gamergate cambiado manera comunicamos inte...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'NO', 'NO']"
4,100005,es,entonces asi mercado mejor hacer cambiarlo seg...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['YES', 'NO', 'YES', 'NO', 'YES', 'YES']"
5,100006,es,aaah si andrew dobson dedico echar mierda game...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'NO', 'NO']"


In [12]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.6 MB/s eta 0:00:00


In [14]:
# Set up tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
#model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


In [15]:
# Tokenize tweets and create attention masks
encoded_tweets = tokenizer.batch_encode_plus(
    df_cleaned['tweet'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

In [16]:
# Define the labels

# Define the mapping for string labels
label_mapping = {'YES': 1, 'NO': 0}

# Convert string representation of lists into lists of strings
string_lists = df_cleaned['labels_task1'].apply(literal_eval)
# print(string_lists.head())

# Convert lists of strings into lists of integers
integer_labels = string_lists.apply(lambda x: [label_mapping[label] for label in x])

# Get the mode (most frequent value) of each list as the label
labels = integer_labels.apply(lambda x: stats.mode(x)[0][0]).to_numpy()

<ipython-input-16-ac049eed8619>:18: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  labels = integer_labels.apply(lambda x: stats.mode(x)[0][0]).to_numpy()


###  additional_data variable, which contains lists of strings for gender and age annotators. To resolve this issue, you can encode the gender and age annotators as numerical values. One way to do this is to use one-hot encoding for both gender and age categories.

In [18]:
gender_dummies = pd.get_dummies(df_cleaned['gender_annotators'].apply(pd.Series).stack()).sum(level=0)
age_dummies = pd.get_dummies(df_cleaned['age_annotators'].apply(pd.Series).stack()).sum(level=0)

# Concatenate the one-hot encoded DataFrames
encoded_annotations = pd.concat([gender_dummies, age_dummies], axis=1)

# Convert each row of the encoded_annotations DataFrame to a tensor
encoded_annotations_list = [torch.tensor(row.values, dtype=torch.float) for _, row in encoded_annotations.iterrows()]

# Add the list of tensors to your dataset
df_cleaned['encoded_annotations'] = encoded_annotations_list

<ipython-input-18-95e563b4752b>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  gender_dummies = pd.get_dummies(df_cleaned['gender_annotators'].apply(pd.Series).stack()).sum(level=0)
<ipython-input-18-95e563b4752b>:3: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  age_dummies = pd.get_dummies(df_cleaned['age_annotators'].apply(pd.Series).stack()).sum(level=0)
<ipython-input-18-95e563b4752b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [19]:
df_cleaned.head()

,id_EXIST,lang,tweet,number_annotators,gender_annotators,age_annotators,labels_task1,encoded_annotations
0,100001,es,ignora capulloel problema youtuber denuncia ac...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['YES', 'YES', 'NO', 'YES', 'YES', 'YES']","[tensor(1.), tensor(0.), tensor(0.), tensor(1.)]"
1,100002,es,si comicsgate parece gamergate pues bien acoso...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'YES', 'NO']","[tensor(1.), tensor(0.), tensor(0.), tensor(1.)]"
2,100003,es,lee gamergate cambiado manera comunicamos inte...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'NO', 'NO']","[tensor(1.), tensor(0.), tensor(0.), tensor(1.)]"
4,100005,es,entonces asi mercado mejor hacer cambiarlo seg...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['YES', 'NO', 'YES', 'NO', 'YES', 'YES']","[tensor(1.), tensor(0.), tensor(0.), tensor(1.)]"
5,100006,es,aaah si andrew dobson dedico echar mierda game...,6,"['F', 'F', 'F', 'M', 'M', 'M']","['18-22', '23-45', '46+', '46+', '23-45', '18-...","['NO', 'NO', 'NO', 'NO', 'NO', 'NO']","[tensor(1.), tensor(0.), tensor(0.), tensor(1.)]"


### CustomDataset class to accept the additional data and include it in the __getitem__ method. It concatenates the additional data with the input embeddings before passing it to the model. 



In [20]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels, additional_data: pd.DataFrame):
        self.encodings = encodings
        self.labels = labels
        self.additional_data = additional_data

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        print(f"Index: {idx}")
        print(f"Additional data: {self.additional_data.loc[idx]}")
        item['additional_data'] = self.additional_data.loc[idx]['encoded_annotations']
        return item

    def __len__(self):
        return len(self.labels)

### Create the dataset with the additional data:

In [21]:
df_cleaned.reset_index(drop=True, inplace=True)
dataset = CustomDataset(encoded_tweets, labels, df_cleaned)


### Split dataset into training and validation sets

In [22]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

### Modifying the model to accept the additional data. Creating a new custom class that inherits from RobertaForSequenceClassification

In [23]:
class CustomRobertaForSequenceClassification(RobertaModel):
    def __init__(self, config: RobertaConfig):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size + 4, config.num_labels)
        
        self.init_weights()

    def forward(self, input_ids, additional_data, attention_mask=None, labels=None):
        outputs = super().forward(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        cls_output = last_hidden_state[:, 0]
        
        combined_output = torch.cat((cls_output, additional_data), dim=1)
        logits = self.classifier(self.dropout(combined_output))
        
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return (loss, logits) if labels is not None else (logits,)


# # Instantiate the model
model = CustomRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
# #model.train()

Some weights of the model checkpoint at roberta-base were not used when initializing CustomRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing CustomRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Define the training arguments

In [24]:
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
)


### using the Hugging Face compute_metrics function to extract accuracy, precision, and other metrics by 

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


### Custom trainer class to pass the additional data to the model

In [30]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        additional_data = inputs.pop("additional_data")
        outputs = model(**inputs, additional_data=additional_data, labels=labels)
        loss = outputs[0] if isinstance(outputs, tuple) else outputs
        return (loss, outputs) if return_outputs else loss

# Set up the custom trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


### Train the model

In [31]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-20-7386c28bb3b1>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Index: 2968
Additional data: id_EXIST                                                          103403
lang                                                                  es
tweet                  diferenciemos trabajo sexual pueden mostrarse ...
number_annotators                                                      6
gender_annotators                         ['F', 'F', 'F', 'M', 'M', 'M']
age_annotators         ['18-22', '23-45', '46+', '46+', '23-45', '18-...
labels_task1                      ['NO', 'NO', 'NO', 'YES', 'YES', 'NO']
encoded_annotations     [tensor(1.), tensor(0.), tensor(0.), tensor(1.)]
Name: 2968, dtype: object
Index: 3380
Additional data: id_EXIST                                                          200217
lang                                                                  en
tweet                                 cant go day without women womening
number_annotators                                                      6
gender_annotators                       

Step,Training Loss
500,0.240200


Streaming output truncated to the last 5000 lines.
Index: 1043
Additional data: id_EXIST                                                          101207
lang                                                                  es
tweet                   ayer volvi mardel hoy pegue doble jornada fotear
number_annotators                                                      6
gender_annotators                         ['F', 'F', 'F', 'M', 'M', 'M']
age_annotators         ['18-22', '23-45', '46+', '46+', '23-45', '18-...
labels_task1                        ['NO', 'NO', 'NO', 'NO', 'NO', 'NO']
encoded_annotations     [tensor(1.), tensor(0.), tensor(0.), tensor(1.)]
Name: 1043, dtype: object
Index: 2018
Additional data: id_EXIST                                                          102304
lang                                                                  es
tweet                  copiado decir igual forma feminismo error divi...
number_annotators                                             

<ipython-input-20-7386c28bb3b1>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Streaming output truncated to the last 5000 lines.
Index: 4475
Additional data: id_EXIST                                                          201457
lang                                                                  en
tweet                  dak bailed pro bowl receivers pick balls back ...
number_annotators                                                      6
gender_annotators                         ['F', 'F', 'M', 'M', 'M', 'F']
age_annotators         ['18-22', '23-45', '18-22', '23-45', '46+', '4...
labels_task1                        ['NO', 'NO', 'NO', 'NO', 'NO', 'NO']
encoded_annotations     [tensor(0.), tensor(1.), tensor(1.), tensor(0.)]
Name: 4475, dtype: object
Index: 1996
Additional data: id_EXIST                                                          102278
lang                                                                  es
tweet                                    aaaaa enseno hacer manspreading
number_annotators                                             

TrainOutput(global_step=912, training_loss=0.2228962496707314, metrics={'train_runtime': 374.4766, 'train_samples_per_second': 38.862, 'train_steps_per_second': 2.435, 'total_flos': 957263886577152.0, 'train_loss': 0.2228962496707314, 'epoch': 3.0})

In [32]:
eval_results = trainer.evaluate()
print(eval_results)

<ipython-input-20-7386c28bb3b1>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Index: 5974
Additional data: id_EXIST                                                          203151
lang                                                                  en
tweet                  love please t-shirts scooped neck busty menopa...
number_annotators                                                      6
gender_annotators                         ['F', 'F', 'M', 'M', 'M', 'F']
age_annotators         ['18-22', '23-45', '18-22', '23-45', '46+', '4...
labels_task1                  ['YES', 'YES', 'YES', 'YES', 'YES', 'YES']
encoded_annotations     [tensor(0.), tensor(1.), tensor(1.), tensor(0.)]
Name: 5974, dtype: object
Index: 4213
Additional data: id_EXIST                                                          201162
lang                                                                  en
tweet                  whatever wants actually that's free woman repr...
number_annotators                                                      6
gender_annotators                       

Streaming output truncated to the last 5000 lines.
Additional data: id_EXIST                                                          202349
lang                                                                  en
tweet                  gross reallywho would possibly ask professor s...
number_annotators                                                      6
gender_annotators                         ['F', 'F', 'M', 'M', 'M', 'F']
age_annotators         ['18-22', '23-45', '18-22', '23-45', '46+', '4...
labels_task1                      ['YES', 'YES', 'NO', 'NO', 'NO', 'NO']
encoded_annotations     [tensor(0.), tensor(1.), tensor(1.), tensor(0.)]
Name: 5274, dtype: object
Index: 3604
Additional data: id_EXIST                                                          200470
lang                                                                  en
tweet                  get mint one pasiko nalang po boss shad see_no...
number_annotators                                                      6
g

In [33]:
model.save_pretrained("/content/model")

In [35]:
!zip -r model.zip "/content/model"

updating: content/model/ (stored 0%)
updating: content/model/config.json (deflated 49%)
updating: content/model/pytorch_model.bin (deflated 12%)


In [37]:
!zip -r results.zip "/content/results"

  adding: content/results/ (stored 0%)
  adding: content/results/runs/ (stored 0%)
  adding: content/results/runs/Apr28_22-24-53_da0fa070f7a9/ (stored 0%)
  adding: content/results/runs/Apr28_22-24-53_da0fa070f7a9/events.out.tfevents.1682723777.da0fa070f7a9.617.6 (deflated 30%)
  adding: content/results/runs/Apr28_22-24-53_da0fa070f7a9/events.out.tfevents.1682720704.da0fa070f7a9.617.0 (deflated 60%)
  adding: content/results/runs/Apr28_22-24-53_da0fa070f7a9/events.out.tfevents.1682723023.da0fa070f7a9.617.4 (deflated 57%)
  adding: content/results/runs/Apr28_22-24-53_da0fa070f7a9/events.out.tfevents.1682720816.da0fa070f7a9.617.2 (deflated 57%)
  adding: content/results/runs/Apr28_22-24-53_da0fa070f7a9/1682720816.188867/ (stored 0%)
  adding: content/results/runs/Apr28_22-24-53_da0fa070f7a9/1682720816.188867/events.out.tfevents.1682720816.da0fa070f7a9.617.3 (deflated 62%)
  adding: content/results/runs/Apr28_22-24-53_da0fa070f7a9/1682720704.1049857/ (stored 0%)
  adding: content/results/